In [2]:
%cd DeepSeek-VL

/Users/User/Desktop/Deep learning/deepseek/DeepSeek-VL


/opt/anaconda3/envs/ml_hw3/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import json
import os
import torch
from tqdm import tqdm
from sklearn.metrics import accuracy_score, roc_auc_score

# Hugging Face / DeepSeek-VL
from transformers import AutoModelForCausalLM
from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images

Python version is above 3.10, patching the collections module.


/opt/anaconda3/envs/ml_hw3/lib/python3.11/site-packages/transformers/models/auto/image_processing_auto.py:594: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


In [4]:
def load_jsonl(path):
    """Load a .jsonl file into a list of dicts."""
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

In [19]:
# ---------------------------------------------------
# 2) Load your dataset
# ---------------------------------------------------
train_data = load_jsonl("data/train.jsonl")
test_data = load_jsonl("data/test.jsonl")
dev_data = load_jsonl("data/dev.jsonl")

In [29]:
# 2) Distribution of labels
from collections import Counter, defaultdict
labels = [ex["label"] for ex in train_data]
label_dist = Counter(labels)
print("Label distribution (0=non-hateful, 1=hateful):")
for lbl, count in label_dist.items():
    print(f"  Label {lbl}: {count} examples")

dev_labels = [ex["label"] for ex in dev_data]
dev_label_dist = Counter(dev_labels)
print("Label distribution (0=non-hateful, 1=hateful):")
for lbl, count in dev_label_dist.items():
    print(f"  Label {lbl}: {count} examples")
# Example output:
# Label 0: 3000 examples
# Label 1: 2000 examples
# 3) Check for same img path but different labels
img_to_labels = defaultdict(set)
for ex in train_data:
    img_to_labels[ex["img"]].add(ex["label"])

duplicates = [(img, list(labels)) for img, labels in img_to_labels.items()
              if len(labels) > 1]

if duplicates:
    print("\nImages with multiple labels in train_data:")
    for img, lbls in duplicates:
        print(f"  {img} => {lbls}")
else:
    print("\nNo images have conflicting labels in train_data.")

Label distribution (0=non-hateful, 1=hateful):
  Label 0: 5450 examples
  Label 1: 3050 examples
Label distribution (0=non-hateful, 1=hateful):
  Label 1: 250 examples
  Label 0: 250 examples

No images have conflicting labels in train_data.


In [44]:
import random

few_shot_examples = train_data[:2]
# Separate train_data by label
non_hateful_data = [ex for ex in train_data if ex["label"] == 0]
hateful_data     = [ex for ex in train_data if ex["label"] == 1]

# 2) Pick 2 from label=0, 1 from label=1
few_shot_non_hateful = random.sample(non_hateful_data, 2)
few_shot_hateful     = random.sample(hateful_data, 1)

# Combine them into a few-shot set (total 3 examples)
few_shot_examples = few_shot_non_hateful + few_shot_hateful
# Optionally shuffle them so the order isn't always the same
random.shuffle(few_shot_examples)
print(few_shot_examples)

[{'id': 57302, 'img': 'img/57302.png', 'label': 1, 'text': 'because they love you they want this not they need to mix breed to preserve their species'}, {'id': 2918, 'img': 'img/02918.png', 'label': 0, 'text': 'a islamic leader among his followers'}, {'id': 23401, 'img': 'img/23401.png', 'label': 0, 'text': "i just realized we know we don't want him in the same bathroom with our wives"}]


In [10]:
model_path = "deepseek-ai/deepseek-vl-1.3B-base"  # or the chat variant
vl_chat_processor = VLChatProcessor.from_pretrained(model_path)
vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True
)
#vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()
vl_gpt = vl_gpt.to(torch.bfloat16).to("mps").eval()


Some kwargs in processor config are unused and will not have any effect: sft_format, mask_prompt, ignore_id, image_tag, num_image_tokens, add_special_token. 


In [25]:
def zero_shot_prompt(example):
    return [
        {
            "role": "User",
            "content": (
                "Classify the following meme as hateful or not hateful.\n\n"
                f"Meme text: {example['text']}\n"
                "At the end of your answer, write:\n"
                "Final Answer: [HATEFUL or NOT HATEFUL]\n"
                "Provide no additional text.\n"
            ),
            "images": [f"data/{example['img']}"],
        },
        {"role": "Assistant", "content": ""},
    ]


In [40]:
def few_shot_prompt(example, few_shot_examples):
    """
    Construct a conversation with a few labeled examples, then ask for the label of the new example.
    """
    # Build the prefix from the few-shot examples
    # We'll just do 2 examples here for brevity:
    prompt_parts = []
    for fs_ex in few_shot_examples:
        label_text = "HATEFUL" if fs_ex["label"] == 1 else "NOT HATEFUL"
        prompt_parts.append(
            f"Meme: <image_placeholder>\n"
            f"Meme text: {fs_ex['text']}\n"
            f"Label: {label_text}\n\n"
        )
    # Combine the few-shot context, then add the new sample
    prompt_str = "".join(prompt_parts)
    prompt_str += (
        f"Meme: <image_placeholder>\n"
        f"Meme text: {example['text']}\n"
        "Question: Is this meme hateful or not?\n"
        "Please answer 'HATEFUL' or 'NOT HATEFUL'."
    )

    # We have multiple images in the conversation: few-shot images, plus the new one
    # We'll line them up in the same order we used in prompt_parts
    images = []
    for fs_ex in few_shot_examples:
        images.append(os.path.join("data", fs_ex["img"]))
    images.append(os.path.join("data", example["img"]))

    conversation = [
        {
            "role": "User",
            "content": prompt_str,
            "images": images,
        },
        {"role": "Assistant", "content": ""},
    ]
    return conversation

def chain_of_thought_prompt(example):
    """
    Construct a conversation that explicitly requests chain-of-thought reasoning,
    then a final label.
    """
    conversation = [
        {
            "role": "User",
            "content": (
                f"Here is a meme: <image_placeholder>\n"
                f"Meme text: {example['text']}\n\n"
                "Please reason step-by-step if it is hateful or not.\n"
                "Then provide the final answer as 'HATEFUL' or 'NOT HATEFUL'.\n"
                "Do not write anything else after your Final Answer.\n"
                "Chain of thought:\n"
            ),
            "images": [os.path.join("data", example["img"])],
        },
        {"role": "Assistant", "content": ""},
    ]
    return conversation

In [47]:
def parse_label_from_output(output_text):
    """
    Parse the model output to decide if it is 'HATEFUL' or 'NOT HATEFUL'.
    Return 1 if 'HATEFUL', 0 if 'NOT HATEFUL'. If uncertain, default to 0.
    """
    # Simple approach: search for these substrings
    # Make them uppercase for easier matching
    out_upper = output_text.upper()
    #print("Response:", out_upper)
    if "HATEFUL" in out_upper and "NOT HATEFUL" not in out_upper:
        return 1
    elif "NOT HATEFUL" in out_upper:
        return 0
    else:
        # No clear label found => default to not hateful
        print("Warning: No clear label found in output.")
        return 0

In [42]:

def run_inference(conversation, processor, model):
    """
    Given a conversation structure, run the DeepSeek-VL model 
    and return the raw text output.
    """
    pil_images = load_pil_images(conversation)
    prepared = processor(
        conversations=conversation,
        images=pil_images,
        force_batchify=True
    ).to(model.device)

    with torch.no_grad():
        inputs_embeds = model.prepare_inputs_embeds(**prepared)
        outputs = model.language_model.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=prepared.attention_mask,
            pad_token_id=processor.tokenizer.eos_token_id,
            eos_token_id=processor.tokenizer.eos_token_id,
            max_new_tokens=50,
            do_sample=False,
            use_cache=True
        )
    decoded = processor.tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
    return decoded

In [34]:
print(preds)
for method, (acc, auroc) in results.items():
    print(f"Method: {method}")
    print(f"  Accuracy: {acc:.4f}")
    print(f"  AUROC:    {auroc:.4f}\n")

[0, 0, 0, 0, 0]
Method: chain_of_thought
  Accuracy: 0.0000
  AUROC:    nan



In [48]:
# ---------------------------------------------------
# 3) Evaluate with each prompting method
# ---------------------------------------------------
methods = [ "few_shot"] #  "zero_shot", "few_shot", "chain_of_thought"
results = {}
cur_test_data = dev_data
for method in methods:
    preds = []
    labels = []
    for example in tqdm(cur_test_data , desc=f"Evaluating {method}"): # test_data
        if method == "zero_shot":
            conversation = zero_shot_prompt(example)
        elif method == "few_shot":
            conversation = few_shot_prompt(example, few_shot_examples)
        else:  # chain_of_thought
            conversation = chain_of_thought_prompt(example)

        output_text = run_inference(conversation, vl_chat_processor, vl_gpt)
        pred_label = parse_label_from_output(output_text)
        preds.append(pred_label)
        labels.append(example["label"])

    # Compute accuracy, auroc
    print("prediction:", preds)
    print("labels:", labels)
    acc = accuracy_score(labels, preds)
    # For AUROC, we need probas or continuous scores. We only have 0/1 from parse_label...
    # In a real scenario, you might try to parse a "confidence" or do a second pass
    # that requests a probability. But here we can do a "binary" interpretation for AUROC:
    try:
        auroc = roc_auc_score(labels, preds)
    except ValueError:
        # If there's only one class in the entire test set, roc_auc_score can fail
        auroc = float("nan")

    results[method] = (acc, auroc)

# ---------------------------------------------------
# 4) Print out the results
# ---------------------------------------------------
for method, (acc, auroc) in results.items():
    print(f"Method: {method}")
    print(f"  Accuracy: {acc:.4f}")
    print(f"  AUROC:    {auroc:.4f}\n")

Evaluating few_shot: 100%|██████████| 500/500 [6:26:25<00:00, 46.37s/it]    

prediction: [1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 